Things to do
- Preprocess images
- Load images
- Set up data for training
- Set up feature learning
- Add classification
- Receive video feed
- Process video feed



In [6]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

### Setting the path to the dataset

In [7]:

import pathlib

data_dir = pathlib.Path("kaggle")

### Creating the dataset




In [ ]:
#TODO: We should consider trying out different batch sizes

batch_size = 32
img_height = 32
img_width = 32